<a href="https://colab.research.google.com/github/smomtahe/Project_BreastCancer_Advanced_DeepLearnig/blob/main/2_Images_CNN_Transformer_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Images are used and trained in the dataset

import pandas as pd
import numpy as np
import zipfile
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, Conv2D, UpSampling2D, concatenate, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load dataset
dataset_df = pd.read_csv("/content/dataImagesSum.csv")

# Extract features (assuming they are in the first 128 columns)
X = dataset_df.iloc[:, 2:130].values
image_paths = dataset_df['path'].values  # Path column

# Specify the path to the zip file
zip_file_path = '/content/images.zip'

# Specify the directory where you want to extract the contents
extracted_dir_path = '/content/images/'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents to the specified directory
    zip_ref.extractall('/content')

print("Extraction complete. The files are extracted to:", extracted_dir_path)


Extraction complete. The files are extracted to: /content/images/


In [ ]:
# Function to load and preprocess images
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)  # Preprocess based on the model you choose
    return img

# Preprocess all images and stack them into a single numpy array
preprocessed_images = np.vstack([preprocess_image(os.path.join(extracted_dir_path, path)) for path in image_paths])


In [ ]:
# Split the dataset into features (X) and targets (Y)
# Setting the first 12 samples as the test set
X_test = X[:12, :]
y_test = preprocessed_images[:12]

# Using the remaining samples as the training set
X_train = X[12:, :]
y_train = preprocessed_images[12:]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam

# Assuming X contains numerical features
X = dataset_df.iloc[:, 2:130].values

# Assuming y contains the image data
y = dataset_df.iloc[:, 130:].values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define simple model architecture
input_features = Input(shape=(128,))
x = Dense(256, activation='relu')(input_features)
x = Dense(512, activation='relu')(x)
output_image = Dense(128, activation='sigmoid')(x)  # Adjusted output size to match the shape of y_train

# Create the model
model = Model(inputs=input_features, outputs=output_image)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Predict images from the test set features
y_pred = model.predict(X_test)

print(f"a=[{', '.join(map(str, y_pred[4]))}];")
print(f"aa=[{', '.join(map(str, y_pred[5]))}];")


1/1 [==============================] - 0s 147ms/step - loss: 0.2281
Test Loss: 0.22806069254875183
1/1 [==============================] - 0s 60ms/step
a=[0.4628131, 0.5822667, 0.5120999, 0.47373757, 0.4990976, 0.52178276, 0.5539396, 0.49500045, 0.49975592, 0.45893002, 0.48761225, 0.37973127, 0.52434266, 0.48505893, 0.44264328, 0.48347217, 0.46523833, 0.47951674, 0.5423786, 0.46079236, 0.538118, 0.46935773, 0.553081, 0.45364383, 0.47372285, 0.5545743, 0.43659794, 0.53957933, 0.53168136, 0.47887823, 0.533823, 0.49411845, 0.4810281, 0.45199138, 0.50742674, 0.45498362, 0.46633697, 0.5745793, 0.4874386, 0.5018254, 0.4652223, 0.437341, 0.56159896, 0.42294347, 0.47027233, 0.4312053, 0.52868533, 0.51716745, 0.5485312, 0.4601754, 0.5746321, 0.6049049, 0.48545945, 0.47573903, 0.5244517, 0.44996837, 0.4905842, 0.5023082, 0.5321399, 0.50742084, 0.4591343, 0.5196516, 0.5442181, 0.52072334, 0.4783967, 0.47118306, 0.55391234, 0.42384055, 0.45152915, 0.4255721, 0.48124248, 0.57895446, 0.4197661, 0.524

'\n\nimport matplotlib.pyplot as plt\n\n# Visualize the predicted images\nfor i in range(2):  # Display the first two predicted images\n    plt.figure()\n    plt.imshow(y_pred[i].reshape(8, 16))  # Assuming each image has a shape of 8x16 pixels\n    plt.title(f"Predicted Image {i+1}")\n    plt.axis(\'off\')\n    plt.show()\n'

In [ ]:
## CNN
import numpy as np
import pandas as pd
from keras.layers import Conv2D, Flatten, Dense, Reshape
from keras.models import Sequential

# Load the dataset
dataset_df = pd.read_csv('/content/dataImagesSum.csv')

# Assuming X contains numerical features and y contains the image data
X = dataset_df.iloc[:, 2:130].values
y = dataset_df.iloc[:, 130:].values

# Reshape y to be in the shape (number_of_samples, image_height, image_width, channels)
# Since the images are likely to be grayscale as per the example, channels will be set to 1
image_height, image_width = 8, 16  # Adjust these dimensions to match your actual image size
y = y.reshape((-1, image_height, image_width, 1))

# Manual split of the dataset
X_test, y_test = X[:12, :], y[:12]
X_train, y_train = X[12:, :], y[12:]

# Define the CNN model architecture
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(128,)))
model.add(Reshape((16, 16, 1)))  # Reshape output to 2D format for convolution layers, adjust size as necessary
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(np.prod(y_train.shape[1:]), activation='sigmoid'))
model.add(Reshape(y_train.shape[1:]))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=1)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Predict images from the test set features
y_pred = model.predict(X_test)


Epoch 1/50
54/54 [==============================] - 4s 16ms/step - loss: 0.2407
Epoch 2/50
54/54 [==============================] - 1s 16ms/step - loss: 0.2312
Epoch 3/50
54/54 [==============================] - 1s 16ms/step - loss: 0.2254
Epoch 4/50
54/54 [==============================] - 1s 16ms/step - loss: 0.2262
Epoch 5/50
54/54 [==============================] - 1s 16ms/step - loss: 0.2167
Epoch 6/50
54/54 [==============================] - 1s 16ms/step - loss: 0.2215
Epoch 7/50
54/54 [==============================] - 1s 16ms/step - loss: 0.2131
Epoch 8/50
54/54 [==============================] - 1s 17ms/step - loss: 0.2127
Epoch 9/50
54/54 [==============================] - 2s 29ms/step - loss: 0.2282
Epoch 10/50
54/54 [==============================] - 2s 32ms/step - loss: 0.2173
Epoch 11/50
54/54 [==============================] - 2s 30ms/step - loss: 0.2153
Epoch 12/50
54/54 [==============================] - 1s 24ms/step - loss: 0.2189
Epoch 13/50
54/54 [==================

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Reshape, UpSampling2D, Conv2DTranspose, LayerNormalization
from tensorflow.keras.models import Model
import tensorflow as tf
from transformers import ViTModel, ViTConfig

# Load the dataset
dataset_df = pd.read_csv('/content/dataImagesSum.csv')

# Assuming X contains numerical features and y contains the image data
# Simulated data for demonstration
X = np.random.rand(88, 32, 32, 3)  # Simulate 88 RGB images of shape 32x32
y = np.random.rand(88, 64, 64, 1)  # Target images are 64x64 grayscale

# Manual split of the dataset
X_test, y_test = X[:12, :], y[:12]
X_train, y_train = X[12:, :], y[12:]

# Load the base model with pre-trained weights
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers on top of the base model
inputs = base_model.input
x = base_model(inputs)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Reshape((8, 8, 4))(x)  # Adjust dimensions as necessary

# Upsample to target size
x = UpSampling2D(size=(8, 8))(x)
x = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(x)  # Use padding='same' to maintain size
x = Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid')(x)  # Final layer for grayscale image

# Create and compile the model
model = Model(inputs, x)
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=1, validation_split=0.1)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Predict images from the test set features
y_pred = model.predict(X_test)

Epoch 1/50
68/68 [==============================] - 13s 54ms/step - loss: 0.0833 - val_loss: 0.0833
Epoch 2/50
68/68 [==============================] - 2s 36ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 3/50
68/68 [==============================] - 2s 24ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 4/50
68/68 [==============================] - 2s 24ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 5/50
68/68 [==============================] - 2s 24ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 6/50
68/68 [==============================] - 2s 23ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 7/50
68/68 [==============================] - 2s 24ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 8/50
68/68 [==============================] - 2s 25ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 9/50
68/68 [==============================] - 2s 36ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 10/50
68/68 [==============================] - 2s 28ms/step - loss: 0.0832 - val_loss: 0.0833
Epoch 11